In [1]:
import torch as pt
import numpy as np
import torch.nn as nn
import time
import matplotlib.pyplot as plt
import random
import os

In [2]:
class lin_class:
    """
    Erzeugt ein iterrierbares Objekt, welches sich mit 1/q schrittweise an die 1 annähert bis die Iteration aufhört

    Dabei gibt es als iterierbares Objekt eine Klasse zurück, die das aktuelle b (wird mit jedem Schritt erhöht) und eine Vergleichsfunktion liefert
    """
    def __init__(self, q):
        self.q = q;
    def __iter__(self):
        class lin_c:
            """
            Erhöht sich bei jedem Schritt um 1/q und gibt diesen Wert zurück
            Sobald b>=1, wird Iteration gestoppt
            """
            def __iter__(self):
                self.b = 0;
                return self
            def __next__(self):
                """
                Falls b < 1, gib eine Funktion und b zurück
                Die Funktion dient dem überprüfen, ob ein x kleiner als ein b ist
                """
                #print("Next")
                # print(self.b)
                if self.b<1:
                    def lin(b, x):
                        """
                        Falls x eine Liste: Gebe Array zurück mit 0 falls x[i]<b, ansonsten x[i]=0
                        Falls x eine Zahl, gebe 0 zurück falls x<b, ansonsten 1

                        Erhöht b um 1/q
                        """
                        ret = []
                        if  isinstance(x,(list)):
                            ret = np.zeros([len(x), ])
                            # print("It's list with length ", len(x))
                            x = np.array(x)
                            for j in range(len(x)):
                                if (x[j]<b):
                                    # print("x =",x, " b=", self.b)
                                    ret[j]=0;
                                else:
                                    # print("x =",x, " b=", self.b)
                                    ret[j]=1;
                                    # print("Going over x!", j)
                            return ret
                        # for j in range(len(x)):
                        if (x<b):
                            # print("x =",x, " b=", self.b)
                            return 0;
                        else:
                            # print("x =",x, " b=", self.b)
                            return 1;
                        # return ret
                    # lin = lambda x: linear(self.b, x)
                    self.b += 1/self.q
                else:
                    raise StopIteration
                return self.b, lin;
            def __init__(self, q):
                self.q = q;
            def __len__(self):
                return self.q
        return lin_c(self.q)

In [3]:
class vclass:
    """
    
    """
    def __init__(self):
        self.b_tot = [];
        # print(self.func_dict)
    def append(self,b, func):
        """
        Fügt der Klasse eine Funktion hinzu und hängt an die Liste aller bs ein b hinzu
        """
        self.func=func;
        self.b_tot.append(b)
        # print(self.func_dict)
    def __iter__(self):
        """
        Gibt ein Element der Klasse vc mit der Liste aller bs und der Funktion zurück
        """
        class vc:
            def __init__(self, b_tot, func):
                self.func=func;
                self.b_tot= b_tot;
                self.count =0;
            def __iter__(self):
                return self;
            def __next__(self):
                """
                Dient dem Durchgehen einer Liste mit allen bs
                
                Falls count noch kleiner als die Länge der Liste an bs, erhöhe den Count

                Gibt das nächste b zurück, mitsamt der Funktion
                """
                if self.count<len(self.b_tot):
                    self.count +=1;
                else:
                    # print("Stop Iteration!")
                    raise StopIteration
                # print(len(self.b_tot), self.count)
                return self.b_tot[self.count-1], self.func
            def __len__(self):
                return len(self.b_tot)
        return vc( self.b_tot, self.func);

In [4]:
def update_V(version_space, x, y):
    """
    """
    v = iter(version_space)
    fs = vclass();
    # print("Update V")
    for b, f in v: # Gehe alle b mit Vergleichsfunktion durch
        if (f(b, x)==y): # Falls b richtig ist, wird dieses b mit der Funktion V hinzugefügt
            fs.append( b, f);
    return fs;
                        
        
def is_in_DIS(version_space, x, num_class):
    """
    Checks if x is smaller and bigger than b from version_space
    """
    count = np.zeros([num_class, ])
    # for i in range(num_class):
    v = iter(version_space)
    for b, f in v:
        # if (f(x)==i):
        # print(b, f)
        count[f(b, x)] +=1
                    
                    
    if (sum(count>0)!=1):
        return 1
    # print(count)
    return 0;
    
    
class generator_uni:
    def draw(self):
        t = np.random.uniform()
        return t

In [5]:
def DoD(version, x_dist, y_query, exp_query, num_class, n_l, n_u, hyp_h, hyp_r):
    #DoD(dclass, unif, y_query, exp_query, num_class, t, n_u, hclass, rclass)
    TOL = 1e-6
    myclass = version
    i =0
    # Solange noch Labels gesamplet werden können/dürfen
    while (i<n_l):
        x = x_dist.draw() #Ziehe eine Zahl zwischen [0,1)
        # print(x)
        if(is_in_DIS(myclass, x, num_class)==1):
            i +=1
            d = int(y_query(x)!=exp_query(x)) # 1 (True), falls x > 0.3 und Experte somit falsch, bei x < 0.3 ist WK 50/50
            # print(x, y_query(x), exp_query(x), y)
            myclass = update_V(myclass, x, d) # Beinhaltet nun V, mit allen b die richtig sind
            v = iter( myclass)
            # print(len(v))
        if (len(iter( myclass))==1):
            break
        
    x_data = []
    y_data = []
    for i in range(n_u):
        x = x_dist.draw()
        x_data.append(x)
        y_data.append(y_query(x))
    # print( y_data)
    errmin = 1;
    # plt.scatter(x_data, h(theta_h, x_data).T)
    list_ret = []
    # x_data = np.array(x_data)
    # y_data = np.array(y_data)
    for theta_d, d in myclass: # Gehe alle b durch
        # print(d)
        # plt.figure()
        # plt.scatter(x_data, d(theta_d, x_data).T)
        # print("Searching!")
        # print(theta_d)
        for theta_h, h in iter(hyp_h): # Durchgehen aller Hypothesen
            for theta_r, r in iter(hyp_r): # Durchgehen aller Rejectoren
                errh = 1*(h(theta_h, x_data)!=np.array(y_data)) # Wo liegt der Klassifikator falsch?
                rej = 1-r(theta_r, x_data) # Rejection berechnen
                nonrej = r(theta_r, x_data)
                ds = d(theta_d, x_data)
                err = np.sum(errh*nonrej+rej*ds)/len(x_data)
                # print(h(theta_h, x_data))
                # print(err)
                # print(y_data)
                # plt.figure()
                # plt.scatter(x_data, h(theta_h, x_data).T)
                # plt.scatter(x_data, y_data)
                if (err<errmin):
                    errmin=err
                    thetah = theta_h
                    thetar = theta_r
                # break
                if err<TOL:
                    # print(theta_d)
                    list_ret.append( (theta_h, h, theta_r, r, theta_d, d))
    if len(list_ret)==0:
        print("No perfect answer!")
        for theta_d, d in myclass:
            for theta_h, h in iter(hyp_h):
                for theta_r, r in iter(hyp_r):
                    errh = 1*(h(theta_h, x_data)!=np.array(y_data))
                    rej = 1-r(theta_r, x_data)
                    nonrej = r(theta_r, x_data)
                    ds = d(theta_d, x_data)
                    err = np.sum(errh*nonrej+rej*ds)/len(x_data)
                    if err-errmin<TOL:
                        # print(theta_d)
                        list_ret.append( (theta_h, h, theta_r, r, theta_d, d))  
    random.shuffle(list_ret)
    return list_ret[0];

In [6]:
def main(param):
    
    # Helps with debuging
    pt.autograd.set_detect_anomaly(True)
    device = pt.device("cpu")
    Size = 1000
    Sizehalf= int(np.floor(Size/2))
    x = pt.rand([Size, 1])
    q = 0;
    qp =0;
    x = pt.sort(x, dim=0).values
    sizexhalf = int(pt.where(x>0.5)[0][0].item())
    y_half = pt.floor(pt.rand([sizexhalf,1] )*2)
    y_other_half = pt.ones([Size-sizexhalf,1] )
    y = pt.vstack((y_half, y_other_half))
    m_half = y_half;
    m_other_half = pt.zeros([Size-sizexhalf,1] )
    m = pt.vstack((m_half, m_other_half))
    # print(y.shape)
    dataloader = pt.hstack((x, y, m))
    # print(dataloader)
    
        
    trainloader=pt.utils.data.DataLoader(dataloader, batch_size=100, shuffle=True, num_workers=8)
    
    
    
    unif = generator_uni() #Zufällige Zahl
    
    
    rand_dict = {}
    def y_query(x):
        """
        - Falls x > 0.3 wird 1 zurückgegeben
        - Falls x <=0.3 wird der Eintrag in rand_dict für x zurückgegeben
            - Falls dieser Eintrag noch nicht existiert, wird zufällig ein Wert 0 oder 1 erzeugt und gespeichert
        """
        if (x>0.3):
            return 1;
        elif (x<=0.3):
            if x not in rand_dict.keys():
                rand_dict[x]= 1*(np.random.uniform()>0.5)
            return rand_dict[x];
    def exp_query(x):
        """
        - Falls x > 0.3 wird 0 zurückgegeben
        - Falls x <=0.3 wird der Eintrag in rand_dict für x zurückgegeben
            - Falls dieser Eintrag noch nicht existiert, wird zufällig ein Wert 0 oder 1 erzeugt und gespeichert
        """
        if (x>0.3):
            return 0;
        elif (x<=0.3):
            if x not in rand_dict.keys():
                rand_dict[x]= 1*(np.random.uniform()>0.5)
            return rand_dict[x];
        
    
    
    def train_ERM(x_dist, y_query, exp_query, num_class, hyp_h, hyp_r, num):
        TOL = 1e-6
        x_data = []
        y_data = []
        exp_data = []
        list_ret = []
        for i in range(num):
            x = x_dist.draw()
            x_data.append(x)
            y_data.append(y_query(x))
            exp_data.append(exp_query(x))
        errt = 10000
        for theta_h, h in iter(hyp_h):
                for theta_r, r in iter(hyp_r):
                    errh = 1*(h(theta_h, x_data)!=np.array(y_data))
                    rej = 1-r(theta_r, x_data)
                    nonrej = r(theta_r, x_data)
                    ds = 1*(np.array(exp_data)!=np.array(y_data))
                    err = np.sum(errh*nonrej+rej*ds)
                    # print(h(theta_h, x_data))
                    # print(err)
                    # print(y_data)
                    # plt.figure()
                    # plt.scatter(x_data, h(theta_h, x_data).T)
                    # plt.scatter(x_data, y_data)
                    
                    if (err<errt):
                        errt=err
                        thetah = theta_h
                        thetar = theta_r
                    # break
                    if err<TOL:
                        # print(theta_d)
                        list_ret.append((theta_h, h, theta_r, r))
        if len(list_ret)!=0:
            random.shuffle(list_ret)
            return list_ret[0];         
        return (thetah, h, thetar, r)
    
    def train_CAL(x_dist, y_query, exp_query, num_class, hyp_h, hyp_r, n_l, n_u):
        TOL = 1e-6
        x_data_u = []
        y_data_u = []
        x_data = []
        y_data = []
        exp_data = []
        list_hr = []
        list_ret = []
        for i in range(n_u):
            x = x_dist.draw()
            
            # if (i<=n_u):
            x_data_u.append(x)
            yq = y_query(x)
            y_data_u.append(yq)
    #         else:
    #             x_data.append(x)
    #             y_data.append(y_query(x))
    #             exp_data.append(exp_query(x))
                
        for theta_h, h in iter(hyp_h):
            for theta_r, r in iter(hyp_r):
                # errh_nl = 1*(h(theta_h, x_data)!=np.array(y_data))
                # nonrej_nl = r(theta_r, x_data)
                errh_nu = 1*(h(theta_h, x_data_u)!=np.array(y_data_u))
                nonrej_nu = r(theta_r, x_data_u)
                err = np.sum(errh_nu*nonrej_nu) 
                # + np.sum(errh_nu*nonrej_nu)
                if err<TOL:
                    list_hr.append((theta_h, h, theta_r, r))
        t = 0            
        # print(len(list_hr))
        while (t<n_l):
            
            # print(t)
            x = x_dist.draw()
            hr = iter(list_hr)
            # print(len(list_hr))
            count = np.zeros([num_class, ])
            
            r_params = []
            for theta_h, h, theta_r, r in hr:
                r_params.append(theta_r)
            if len(set(r_params))==1:
                print("Only 1 rejector is remained", set(r_params))
                break;
            # print(min(r_params))
            # print("Passed finding rejectors. Number of rejectors are "+str(len(set(r_params))))
            hr = iter(list_hr)
            for theta_h, h, theta_r, r in hr:
                # print("HR
                count[r(theta_r, x)] +=1
                if (sum(count>0)!=1):
                    yq = y_query(x)
                    expq = exp_query(x)
                    # print(len(list_hr))
                    iter_hr = iter(list_hr)
                    list_hr = [(theta_hh, hh, theta_rr, rr)for theta_hh, hh, theta_rr, rr in iter_hr  if ((r(theta_rr, x)==0 and yq == expq) or (r(theta_rr, x)==1))]
                    t +=1
                    # print(t, len(list_hr))
                    break;
                    
            # print(count)
        # print("t=", t)
        # print(len(list_hr))
        if len(list_hr)!=0:
            random.shuffle(list_hr)
            # print("Return")
            # print(len(list_hr))
            return list_hr[0];   
        print("No list exists")
        return 0
            
    
    def train_staged_ERM(x_dist, y_query, exp_query, num_class, hyp_h, hyp_r, n_u, n_l):
        myclass = hyp_h
        TOL = 1e-6
        x_data_u = []
        y_data_u = []
        x_data = []
        y_data = []
        exp_data = []
        list_ret = []
        hlist = []
        for i in range(n_u):
            x = x_dist.draw()
            x_data_u.append(x)
            yq = y_query(x)
            y_data_u.append(yq)
            
        # Find the classifiers with minimum error
        errmin = 1;
        for theta_h, h in iter(hyp_h):
            errh = 1*(h(theta_h, x_data_u)!=np.array(y_data_u))
            err = np.sum(errh)/len(x_data_u)
            if (err<errmin):
                errmin = err
                # break
                
        for theta_h, h in iter(hyp_h):
            errh = 1*(h(theta_h, x_data_u)!=np.array(y_data_u))
            err = np.sum(errh)/len(x_data_u)
            if err-errmin<TOL:
                # print(theta_d)
                hlist.append((theta_h, h))
        # print("Length of classifier hypothesis class is ", len(hlist))
        for i in range(n_l):
            x = x_dist.draw()
            x_data.append(x)
            y_data.append(y_query(x))
            exp_data.append(exp_query(x))
                
        errmin = 1
        for theta_h, h in iter(hlist):
                for theta_r, r in iter(hyp_r):
                    errh = 1*(h(theta_h, x_data)!=np.array(y_data))
                    rej = 1-r(theta_r, x_data)
                    nonrej = r(theta_r, x_data)
                    ds = 1*(np.array(exp_data)!=np.array(y_data))
                    err = np.sum(errh*nonrej+rej*ds)/len(x_data)
                    # print(h(theta_h, x_data))
                    # print(err)
                    # print(y_data)
                    # plt.figure()
                    # plt.scatter(x_data, h(theta_h, x_data).T)
                    # plt.scatter(x_data, y_data)
                    
                    if (err<errmin):
                        errmin=err
                    # break
                    if err<TOL:
                        # print(theta_d)
                        list_ret.append((theta_h, h, theta_r, r))
        if len(list_ret)==0:
            for theta_h, h in iter(hlist):
                for theta_r, r in iter(hyp_r):
                    errh = 1*(h(theta_h, x_data)!=np.array(y_data))
                    rej = 1-r(theta_r, x_data)
                    nonrej = r(theta_r, x_data)
                    ds = 1*(np.array(exp_data)!=np.array(y_data))
                    err = np.sum(errh*nonrej+rej*ds)/len(x_data)
                    if err-errmin<=TOL:
                        # print(theta_d)
                        list_ret.append((theta_h, h, theta_r, r))
        random.shuffle(list_ret)
        return list_ret[0];  
        # return (thetah, h, thetar, r)
                
        
        
            
    def test_def(x_dist, y_query, exp_query, theta_h, h, theta_r, r, num):
        x_data = []
        y_data = []
        m_data = []
        for i in range(num):
            x = x_dist.draw()
            x_data.append(x)
            y = y_query(x)
            m = exp_query(x)
            # if (x<0.3):
            #     print(m, y)
            y_data.append(y)
            m_data.append(m)
        errh = 1*(h(theta_h, x_data)!=np.array(y_data))
        # print(np.sum(errh)/num)
        rej = 1-r(theta_r, x_data)
        nonrej = r(theta_r, x_data)
        ds = 1*(np.array(m_data)!=np.array(y_data))
        # print(m_data, y_data, ds)
        err = np.sum(errh*nonrej+rej*ds)
        return err/num
    
    
    num_class = 2
    range_labels = param["range-labels"]
    ITERS = param["iters"]
    test_err =np.zeros([range_labels,])
    test_err_ERM = np.zeros([range_labels,])
    test_err_CAL =np.zeros([range_labels,])
    test_err_staged_ERM = np.zeros([range_labels,])
    n_test = 1000
    n_u = 100
    quant_models = 100
    # Für t ist Nummer des Labels bis range_labels=Maximale Anzahl an Labels die gesamplet werden
    for t in range(1, range_labels+1):
        print("T= ", t)
        # Gehe alle Iterationen durch
        #for p in range(ITERS):
        #    hclass = lin_class(quant_models) # Für den Klassifikator
        #    rclass = lin_class(quant_models) #Für den Rejector
        #    theta_h, h, theta_r, r = train_ERM(unif, y_query, exp_query, num_class, hclass, rclass, t)
            # print(theta_h, theta_r, theta_d)
        #    test_err_ERM[t-1] += test_def(unif, y_query, exp_query, theta_h, h, theta_r, r,  n_test)
        for p in range(ITERS):
            dclass = lin_class(quant_models)
            hclass = lin_class(quant_models)
            rclass = lin_class(quant_models)
            theta_h, h, theta_r, r, theta_d, d = DoD(dclass, unif, y_query, exp_query, num_class, t, n_u, hclass, rclass)
            # print(theta_h, theta_r, theta_d)
            test_err[t-1] += test_def(unif, y_query, exp_query, theta_h, h, theta_r, r,  n_test)
        
        for p in range(ITERS):
            
            hclass = lin_class(quant_models)
            rclass = lin_class(quant_models)
            theta_h, h, theta_r, r = train_staged_ERM(unif, y_query, exp_query, num_class, hclass, rclass, n_u, t)
            
            # print(theta_h, theta_r, theta_d)
            test_err_staged_ERM[t-1] += test_def(unif, y_query, exp_query, theta_h, h, theta_r, r,  n_test)
        for p in range(ITERS):
            hclass = lin_class(quant_models)
            rclass = lin_class(quant_models)
            theta_h, h, theta_r, r = train_CAL( unif, y_query, exp_query, num_class, hclass, rclass, t, n_u)
            # print(theta_h, h, theta_r, r)
            # print(theta_h, theta_r, theta_d)
            test_err_CAL[t-1] += test_def(unif, y_query, exp_query, theta_h, h, theta_r, r,  n_test) 

In [ ]:
param = {
    "iters": 10000,
    "range-labels": 20,
}

main(param)

T=  1


/home/joli/joli-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
